In [134]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import sys
from lxml import html

In [2]:
response = requests.get('https://www.baseball-reference.com/teams/tgl.cgi?team=PHI&t=b&year=2020')

In [7]:
soup = BeautifulSoup(response.content, "lxml")

In [10]:
table = soup.find('table')

In [159]:
pd.set_option('display.max_rows', 15)


In [160]:
data = pd.read_html(str(table))[0]
data.rename(columns={'Unnamed: 3':'Home/Away', 'Thr':'Opp Starter Thr', '#':'Players Used'}, inplace=True)
data.drop(data[data['OBP'] == 'OBP'].index, inplace=True)
data.replace({'Home/Away': {'@':'A'}}, inplace=True)
data['Home/Away'].fillna('H', inplace=True)
data[['Result', 'Runs Against']] = data['Rslt'].str.split(',', expand=True)
data['Runs Against'] = data['Runs Against'].str.split('-').str[1]
data['Date'] = data['Date'].str.slice(stop=6)
data.drop(columns=['Rslt', 'Rk', 'Gtm', 'Opp. Starter (GmeSc)'], inplace=True)
game_ids = [hash(data['Date'].iloc[i] + ((data['Opp'].iloc[i] + 'PHI') if data['Home/Away'].iloc[i] == 'H' else ('PHI' + data['Opp'].iloc[i])) + str(data['R'].iloc[i] + data['Runs Against'].iloc[i])) % 2147483647 for i in range(len(data.index))]
data['game_id'] = game_ids
team_code = 'PHI'
team_ids = [sum([ord(char) for char in team_code]) for i in range(len(data.index))]
data['team_id'] = team_ids
data.set_index('game_id', inplace=True)
data = data[['team_id', 'Date', 'Home/Away', 'Opp', 'Result', 'Runs Against', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF',
 'ROE', 'GDP', 'SB', 'CS', 'LOB', 'Players Used', 'BA', 'OBP', 'SLG', 'OPS', 'Opp Starter Thr']]
data.loc[:, 'Runs Against':'Players Used'] = data.loc[:, 'Runs Against':'Players Used'].apply(pd.to_numeric)
data.loc[:, 'BA':'OPS'] = data.loc[:, 'BA':'OPS'].apply(pd.to_numeric)
data.sort_values(by=['game_id'])

,team_id,Date,Home/Away,Opp,Result,Runs Against,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,ROE,GDP,SB,CS,LOB,Players Used,BA,OBP,SLG,OPS,Opp Starter Thr
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8370505,225,Aug 9,H,ATL,L,5,30,23,2,3,1,0,0,2,6,0,3,0,0,1,0,0,1,0,7,11,0.236,0.342,0.420,0.762,R
13264862,225,Jul 24,H,MIA,L,5,34,31,2,5,0,0,1,1,3,0,8,0,0,0,1,2,0,0,5,9,0.161,0.235,0.258,0.493,R
34443936,225,Aug 30,H,ATL,L,12,42,39,10,13,2,1,4,10,2,0,8,1,0,0,0,1,0,0,5,9,0.262,0.344,0.462,0.806,L
39306724,225,Aug 6,H,NYY,W,4,35,31,5,8,2,1,1,5,4,0,8,0,0,0,0,0,0,0,6,10,0.243,0.346,0.432,0.779,L
43211941,225,Aug 29,H,ATL,W,1,34,28,4,7,2,0,1,4,2,0,4,2,1,1,0,1,1,0,6,11,0.259,0.343,0.450,0.793,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051519245,225,Aug 20,A,TOR,L,3,28,26,2,6,0,0,1,2,1,0,11,1,0,0,0,1,1,0,5,9,0.259,0.341,0.459,0.800,R
2058099105,225,Aug 20,A,TOR,L,9,34,29,8,9,3,0,0,8,3,0,9,1,0,1,0,0,0,0,5,11,0.261,0.343,0.457,0.800,R
2098089197,225,Aug 16,H,NYM,W,2,38,37,6,14,2,1,1,5,1,1,8,0,0,0,1,1,0,0,8,11,0.261,0.343,0.460,0.803,R


In [139]:
tree = html.fromstring(response.content)
tree.xpath('//div/div/div/div[contains(@data-template, \'Partials/Teams/Summary\')]/p[strong[contains(text(), \'Record\')]]/a/text()')[0]

'NL_East'

In [ ]:
def insert_batting(conn, data, game_id):
    batting_query = []
    for team in ['home', 'away']:
        batting_data = data[team]['teamStats']['batting']
        singles = batting_data['hits'] - (batting_data['doubles'] + batting_data['triples'] + batting_data['homeRuns'])
        for stat in ['avg', 'obp', 'slg', 'ops', 'stolenBasePercentage']:
            batting_data[stat] = float(batting_data[stat]) if batting_data[stat] != '.---' else None
        batting_data = list(batting_data.values())[:-1]
        batting_query.append((game_id, data[team]['team']['id'], *batting_data, singles))
    # Insert into Batting table for game
    batting_box_score = '''
    INSERT INTO BattingGame (
        game_id,
        team_id,
        fly_outs,
        ground_outs,
        runs,
        doubles,
        triples,
        home_runs,
        strike_outs,
        walks,
        intentional_walks,
        hits,
        hit_by_pitch,
        BA,
        AB,
        OBP,
        SLG,
        OPS,
        caught_stealing,
        bases_stolen,
        stolen_base_percentage,
        ground_into_double_play,
        ground_into_triple_play,
        plate_appearances,
        total_bases,
        RBI,
        LOB,
        sac_bunts,
        sac_flies,
        catchers_interference,
        pickoffs,
        singles)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
    try:
        conn.cursor().executemany(batting_box_score, batting_query)
        conn.commit()
    except sqlite3.IntegrityError:
        print('Game already added')
    except Exception as e:
        db_error_cleanup(conn, e)


def insert_pitching(conn, data, game_id):
    pitching_query = []
    for team in ['home', 'away']:
        pitching_data = data[team]['teamStats']['pitching']
        singles = pitching_data['hits'] - (pitching_data['doubles'] + pitching_data['triples'] + pitching_data['homeRuns'])
        for stat in ['obp', 'era', 'inningsPitched', 'stolenBasePercentage', 'whip']:
            pitching_data[stat] = float(pitching_data[stat]) if pitching_data[stat] != '.---' else None
        for stat in ['hitBatsmen', 'groundOutsToAirouts', 'runsScoredPer9', 'homeRunsPer9']:
            pitching_data.pop(stat)
        pitching_data = list(pitching_data.values())
        pitching_query.append((game_id, data[team]['team']['id'], *pitching_data, singles))
    # Insert into Pitching table for game
    pitching_box_score = '''
    INSERT INTO PitchingGame (
        game_id,
        team_id,
        ground_outs,
        air_outs,
        runs,
        doubles,
        triples,
        home_runs,
        strike_outs,
        walks,
        intentional_walks,
        hits,
        hit_by_pitch,
        AB,
        OBP,
        caught_stealing,
        stolen_bases,
        stolen_base_percentage,
        ERA,
        IP,
        save_oppurtunities,
        earned_runs,
        WHIP,
        batter_faced,
        outs,
        complete_games,
        shutouts,
        balks,
        wild_pitches,
        pickoffs,
        RBI,
        inherited_runners,
        inherited_runners_scored,
        catchers_interference,
        sac_bunts,
        sac_flies,
        singles)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
    try:
        conn.cursor().executemany(pitching_box_score, pitching_query)
        conn.commit()
    except sqlite3.IntegrityError:
        print('Game already added')
    except Exception as e:
        db_error_cleanup(conn, e)

In [146]:
import sqlite3

conn = sqlite3.connect('TestDB1.db')
c = conn.cursor()

c.execute('CREATE TABLE CARS (Brand text, Price number)')
conn.commit()

Cars = {'Brand': ['Honda Civic','Toyota Corolla','Ford Focus','Audi A4'],
        'Price': [22000,25000,27000,35000]
        }

df = pd.DataFrame(Cars, columns= ['Brand', 'Price'])
df.to_sql('CARS', conn, if_exists='replace', index = False)